In [1]:
# Library
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import os, cv2, glob, random
import pandas as pd

from numpy import asarray, clip
from PIL import Image

from tensorflow.keras.utils import normalize
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.optimizers import adam_v2
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, ZeroPadding2D
from keras.layers import Dropout, Lambda, Activation, MaxPool2D, Concatenate, AveragePooling2D, Multiply, Add, Subtract
from keras.models import Model, Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2, l1

In [ ]:
# Unet using VGG-19 into Bloc2
def conv_block(input, num_filters): 
    x = Conv2D(num_filters, 3, padding='same',kernel_initializer = 'he_normal',
               kernel_regularizer=l2(0.00005),bias_regularizer=l2(1e-4))(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(num_filters, 3, padding='same',kernel_initializer = 'he_normal',
              kernel_regularizer=l2(0.00005),bias_regularizer=l2(1e-4))(x)  
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x
    
def conv_1block(input, num_filters): 
    x = Conv2D(num_filters, 3, padding='same',kernel_initializer = 'he_normal',
               kernel_regularizer=l2(0.00005),bias_regularizer=l2(1e-4))(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    return x

# Encoder Block Conv Block followed by maxpooling, return both convolution and maxpooling outputs.The conv output can be 
# used for concatenation (skip connections) with decoder
def encoder_1block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPooling2D((2,2))(x)
    return x, p

def encoder_2block(input, num_filters):
    x = conv_block(input, num_filters)
    x = conv_block(x, num_filters)
    p = MaxPooling2D((2,2))(x)
    return x, p

# Decoder Block skip features gets input from encoder for concatenation
def decoder_1block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters,(2,2), strides=2, padding='same', kernel_initializer = 'he_normal',
               kernel_regularizer=l2(0.00005),bias_regularizer=l2(1e-4))(input)
    x = conv_1block(x, num_filters)  
    return x
def decoder_1Rblock(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same',kernel_initializer = 'he_normal',
               kernel_regularizer=l2(0.00005),bias_regularizer=l2(1e-4))(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x
    
# Decoder to residu Block skip features gets input from encoder for concatenation
def decoder_2Rblock(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same',kernel_initializer = 'he_normal',
               kernel_regularizer=l2(0.00005),bias_regularizer=l2(1e-4))(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    x = conv_1block(x, num_filters)
    return x

In [ ]:
# Build U-Net using the Residu Block
def build_U_VGG19(input_shape, n_classes):
    inputs = Input(input_shape)
    
    c1, p1 = encoder_1block(inputs, 32)  #32
    c2, p2 = encoder_1block(p1, 32)   #64
    c3, p3 = encoder_2block(p2, 64)  #128
    c4, p4 = encoder_2block(p3, 256)  #256
    c5, p5 = encoder_2block(p4, 256)  #256
    
    b1 = conv_block(p5, 512) #bridge
            
    d1 = decoder_2Rblock(b1, c5, 128)  #128
    d2 = decoder_2Rblock(d1, c4, 64)  #128
    d3 = decoder_1Rblock(d2, c3, 32)   #32
    d4 = decoder_1block(d3, c2, 32)    #32
    d5 = decoder_1block(d4, c1, 16)    #16
    
    if n_classes == 1 :
        activation = 'sigmoid'
    else:
        activation = 'softmax'
    
    outputs = Conv2D(n_classes, 1, padding = 'same', activation = activation)(d5) 
    print(activation)
    
    model = Model(inputs, outputs, name = 'M1-VGG-19')
    return model

In [ ]:
# compile mode to 3 kelas RGB
model2 = build_U_VGG19(input_shape = (512 ,512, 3), n_classes=3)

model2.compile(optimizer='adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])
model2.summary()

# save the model for future use to 4 kelas
model2.save('data/model_save/MVGG19_Baru+3RGB_PB_512.hdfs')

In [ ]:
# compile mode to 3 kelas RGB
model2 = build_U_VGG19(input_shape = (1024 ,1024, 3), n_classes=3)

model2.compile(optimizer='adam', loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ['accuracy'])
model2.summary()

# save the model for future use to 4 kelas
model2.save('data/model_save/MVGG19_Baru+3RGB_PB_1024.hdfs')